In [1]:

import json 
import threading
import time
from joblib import load

from flask import Flask, render_template
from flask_cors import CORS

from api_scripts.binance import get_binance_prices 
from api_scripts.coin_gecko import get_coingecko_prices 
from api_scripts.coin_layer import get_coinlayer_prices 

In [2]:

def format_cgd(cgd):
    dataf = {}
    for coin in cgd:
        if(coin["Symbol"]=="btc"):
             dataf["BTC"] = coin["Price"]
        if(coin["Symbol"]=="ltc"):
             dataf["LTC"] = coin["Price"]
        if(coin["Symbol"]=="ada"):
             dataf["ADA"] = coin["Price"]
        if(coin["Symbol"]=="eth"):
             dataf["ETH"] = coin["Price"]
        if(coin["Symbol"]=="usdt"):
             dataf["USDT"] = coin["Price"]
    
    return dataf
    
def format_bd(bd):
        return {"BTC":float(bd["BTCUSD"]),"LTC":float(bd["LTCUSD"]),"ADA":float(bd["ADAUSD"]),"ETH":float(bd["ETHUSD"]),"USDT":float(bd["USDTUSD"]),}

def opportunity(buy,price,sell,cost,coin):
    return{"coin":coin,
    "sellerMarket":buy,
    "buyPrice":price,
    "buyerMarket":sell,
    "sellPrice":cost }
    
def compare(m1,m2,coin):
    m1price = m1["data"][coin]
    m2price = m2["data"][coin]
    if(m1price != m2price): 
        if(m1price< m2price): 
            return opportunity(m1["Name"],m1price,m2["Name"],m2price,coin)
        else:
            return opportunity(m2["Name"],m2price,m1["Name"],m1price,coin)
    else: 
        return False
    
def generate_arbitrage_opportunities(live):
    if live == False:
        print("stale")
        with open("jsons/cgarb.json","r") as f:
            cgd = json.load(f)
        with open("jsons/barb.json","r") as f:
            bd = json.load(f)
        with open("jsons/clarb.json","r") as f:
            clfd = json.load(f)["rates"]
    else:
        cgd = get_coingecko_prices()
        bd = get_binance_prices()
        clfd = json.loads(get_coinlayer_prices())["rates"]

    clf = {"Name":"CoinLayer","data":clfd}
    cgf = {"Name":"CoinGecko","data":format_cgd(cgd)}
    bf = {"Name":"Binance","data":format_bd(bd)}

    opportunities = []
    for coin in clf["data"]:
        comps = [ compare(clf,cgf,coin),
             compare(clf,bf,coin),
             compare(cgf,bf,coin)]
            
        for comp in comps:
            if comp != 0:
                opportunities.append(comp)
    return opportunities

def retrieveModels():
    coins = ['btc','ltc','eth','ada','usdt']
    mls = {}

    for coin in coins:
        mls[coin] = {"model":load(f"ml/models/model_{coin}.joblib"),
        "regressor":load(f"ml/regressors/regressor_{coin}.joblib")}
    print(mls)    
    return mls

mlss = retrieveModels()
btcm = mlss["btc"]["model"]

print(btcm)
btcm.

def model_preds():
    print("hey model_predd","no")
    return "no"


{'btc': {'model': LinearRegression(), 'regressor': LinearRegression()}, 'ltc': {'model': LinearRegression(), 'regressor': LinearRegression()}, 'eth': {'model': LinearRegression(), 'regressor': LinearRegression()}, 'ada': {'model': LinearRegression(), 'regressor': LinearRegression()}, 'usdt': {'model': LinearRegression(), 'regressor': LinearRegression()}}
LinearRegression()


AttributeError: 'LinearRegression' object has no attribute 'info'

In [4]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})

data2 = {"data":generate_arbitrage_opportunities(False)}

def get_data_five_minutes():
    while True:
        data2["data"] = generate_arbitrage_opportunities(True)
        time.sleep(300) 

@app.route('/')
def home():
   return render_template('index.html',data=data2["data"])

@app.route('/learn')
def a():
    return render_template('learn.html')

@app.route('/predictions')
def b():
    return render_template('predictions.html',data=json.dumps(model_preds()))

if(__name__ == "__main__"):
    t = threading.Thread(target=get_data_five_minutes)
    t.start()
    app.run()

stale
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2023 12:47:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 13:14:29] "GET / HTTP/1.1" 200 -
